<a href="https://colab.research.google.com/github/Tharun-C0/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers torch accelerate gradio sympy


In [ ]:
import torch
import re
import gradio as gr
import sympy as sp

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [ ]:
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


In [ ]:
def is_simple_math(expr):
    return bool(re.fullmatch(r"[0-9+\-*/(). ]+", expr))


In [ ]:
def solve_math_exact(expr):
    try:
        result = sp.sympify(expr, evaluate=True)
        return result
    except:
        return None


In [ ]:
def explain_math(expr, result):
    prompt = f"""
    Explain how to solve the following math problem step by step.
    Do NOT calculate the final answer.
    The final answer is already provided.

    Problem: {expr}
    Final Answer: {result}

    Explanation:
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=160,
        temperature=0.2,
        do_sample=False
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
def is_definition_question(text):
    keywords = [
        "define", "definition", "what is", "explain",
        "meaning of", "describe", "tell me about"
    ]
    return any(k in text.lower() for k in keywords)


In [ ]:
conversation_history = []

def chatbot_response(user_input):


    if is_simple_math(user_input):
        exact_result = solve_math_exact(user_input)

        if exact_result is not None:
            explanation = explain_math(user_input, exact_result)

            return (
                f"🧮 **Math Solution**\n\n"
                f"Expression: {user_input}\n"
                f"Final Answer: {exact_result}\n\n"
                f"Explanation:\n{explanation}"
            )


    if is_definition_question(user_input):
        prompt = f"""
        You are an expert computer science professor.
        Give a clear, accurate, and well-structured explanation.
        Use simple language and examples if needed.

        Question: {user_input}
        Answer:
        """
    else:

        context = " ".join(conversation_history[-4:])
        prompt = f"""
        You are a friendly and intelligent AI assistant.
        Maintain a natural conversation.

        Conversation history:
        {context}

        User: {user_input}
        Assistant:
        """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.3,
        do_sample=False
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    conversation_history.append(f"User: {user_input}")
    conversation_history.append(f"Assistant: {response}")

    return response


In [ ]:
def chat_ui(user_message, history):
    bot_reply = chatbot_response(user_message)
    history.append((user_message, bot_reply))
    return history, ""


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("THARUN  Chatbot")
    gr.Markdown("Handles conversation, definitions, and **accurate step-by-step math reasoning**")

    chatbot = gr.Chatbot(height=450)
    msg = gr.Textbox(placeholder="Ask anything (math, concepts, chat)...")
    clear = gr.Button("Clear Chat")

    msg.submit(chat_ui, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: [], None, chatbot)

demo.launch(share=False)
